# Medium amount of cargo and trips

Delivery scenario:
* 1 depot
* 11 to 25 vehicles
* and 15–30 customer locations

## Deciding number of vehicls and costumers

In [19]:
import csv
import random

# Grab random amount
num_vehicles = random.randint(11, 25)
customers = random.randint(15, 30)

print(f"Number of Vehicles = {num_vehicles}")
print(f"Number of Customers = {customers}")


Randomly chosen 24 customer locations:
Customer ID: 16, Location: (80.71282732743802, 72.9731786693818)
Customer ID: 21, Location: (70.45718362149235, 4.5824383655662215)
Customer ID: 48, Location: (6.352770615195713, 38.16192865065368)
Customer ID: 20, Location: (86.17069003107773, 57.735214525676206)
Customer ID: 46, Location: (10.964913035065916, 62.744604170309)
Customer ID: 39, Location: (91.45475897405436, 45.88518525873988)
Customer ID: 34, Location: (22.904807196410438, 3.210024390403776)
Customer ID: 24, Location: (10.100142940972912, 27.79736031100921)
Customer ID: 29, Location: (17.1138648198097, 72.91267979503492)
Customer ID: 10, Location: (80.94304566778267, 0.6498759678061017)
Customer ID: 9, Location: (22.044062204069668, 58.92656838759087)
Customer ID: 19, Location: (82.94046642529949, 61.85197523642461)
Customer ID: 32, Location: (55.694974377464625, 68.46142509898746)
Customer ID: 44, Location: (99.75376064951102, 50.95262936764645)
Customer ID: 2, Location: (27.5029

In [20]:
# Grabbing random locations from csv
with open("customers.csv","r") as f:
    reader = csv.reader(f)
    skip_header = next(reader)
    locations = [(int(row[0]), float(row[1]), float(row[2])) for row in reader if row]
    customer_locations = random.sample(locations, customers)

print(f"Randomly chosen {customers} customer locations:")

medium_cargo_trip = []

for loc in customer_locations:

    # insert coordinates into medium_cargo_trip
    coordinates = (loc[1], loc[2])
    medium_cargo_trip.append(coordinates)
    print(f"Customer ID: {loc[0]}, Location: {coordinates}")


[(80.71282732743802, 72.9731786693818), (70.45718362149235, 4.5824383655662215), (6.352770615195713, 38.16192865065368), (86.17069003107773, 57.735214525676206), (10.964913035065916, 62.744604170309), (91.45475897405436, 45.88518525873988), (22.904807196410438, 3.210024390403776), (10.100142940972912, 27.79736031100921), (17.1138648198097, 72.91267979503492), (80.94304566778267, 0.6498759678061017), (22.044062204069668, 58.92656838759087), (82.94046642529949, 61.85197523642461), (55.694974377464625, 68.46142509898746), (99.75376064951102, 50.95262936764645), (27.502931836911927, 22.321073814882276), (9.090941217379388, 4.711637542473457), (7.9791976923627495, 23.27908863610302), (84.28519201898096, 77.59999115462448), (56.13681341631508, 26.274160852293527), (95.72130722067811, 33.65945451126267), (64.98844377795233, 54.49414806032167), (22.789827565154685, 28.938796360210716), (37.853437720835345, 55.2040631273227), (98.95233506365952, 63.999975985409286)]


In [ ]:
print(medium_cargo_trip)

In [ ]:
import math, random, time, json, os, csv
from dataclasses import dataclass
from typing import List, Tuple
import numpy as np
import pandas as pd

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
random.seed(42)
np.random.seed(42)
plt.rcParams['figure.figsize'] = (6,4)

In [ ]:
GENERATIONS = 5

In [ ]:
def euclidean_distance(a: Tuple[float, float], b: Tuple[float, float]) -> float:
    # same API, inline sqrt style
    return ((a[0] - b[0])**2 + (a[1] - b[1])**2) ** 0.5


def distance_matrix(coords: List[Tuple[float, float]]) -> np.ndarray:
    n = len(coords)
    D = np.zeros((n, n), dtype=float)
    for i in range(n):
        xi, yi = coords[i]
        for j in range(i + 1, n):
            xj, yj = coords[j]
            d = ((xi - xj)**2 + (yi - yj)**2) ** 0.5
            D[i, j] = d
            D[j, i] = d
    return D


## VRP Instances

In [ ]:
@dataclass
class VRPInstance:
    name: str
    depot: Tuple[float,float]
    customers: List[Tuple[float,float]]
    vehicles: int

def build_D(inst: VRPInstance):
    coords = [inst.depot] + inst.customers
    return distance_matrix(coords)


## Encoding and Fittness

In [ ]:

def split_equal_chunks(perm: List, k: int) -> List[List[int]]:
    n = len(perm)
    if k <= 0: return [perm]
    base = n // k
    rem = n % k
    chunks = []
    idx = 0
    for i in range(k):
        size = base + (1 if i < rem else 0)
        chunks.append(perm[idx: idx+size])
        idx += size
    return chunks

def fitness(perm: List[int], k: int, D: np.ndarray) -> float:
    routes = split_equal_chunks(perm, k)
    total = 0.0
    for r in routes:
        if not r: 
            continue
        total += D[0, r[0]]
        for i in range(len(r)-1):
            total += D[r[i], r[i+1]]
        total += D[r[-1], 0]
    return total


## GA - Crossover, mutate, selection

In [ ]:

def crossover(p1: List[int], p2: List[int]) -> List[int]:
    n = len(p1)
    a, b = sorted(random.sample(range(n), 2))
    child = [None]*n
    child[a:b+1] = p1[a:b+1]
    p2_iter = [x for x in p2 if x not in child]
    idxs = list(range(0,a)) + list(range(b+1,n))
    for idx, val in zip(idxs, p2_iter):
        child[idx] = val
    return child

def mutate(ind: List[int], pm: float) -> List[int]:
    ind = ind[:]
    n = len(ind)
    for i in range(n):
        if random.random() < pm:
            j = random.randrange(n)
            ind[i], ind[j] = ind[j], ind[i]
    return ind

def _tournament_select(pop: List[List[int]], fits: List[float], tsize: int = 3) -> List[int]:
    best = None
    for _ in range(tsize):
        i = random.randrange(len(pop))
        if (best is None) or (fits[i] < fits[best]):
            best = i
    return pop[best]


## GA Loop

In [ ]:
def genetic_algorithm(D: np.ndarray,
                      k: int,
                      n_customers: int,
                      params: Dict[str, Any],
                      seed: int = 0,
                      max_seconds: float = None) -> Dict[str, Any]:
    rnd_state = random.getstate()
    np_state = np.random.get_state()

    random.seed(seed)
    np.random.seed(seed)

    pop_size = params.get("pop", 100)
    gens     = params.get("gens", 200)
    pc       = params.get("pc", 0.9)
    pm       = params.get("pm", 0.1)
    tsize    = params.get("tsize", 3)

    customers = list(range(1, n_customers + 1))  # depot is 0
    pop = [random.sample(customers, len(customers)) for _ in range(pop_size)]
    fits = [fitness(ind, k, D) for ind in pop]

    best_idx = min(range(len(fits)), key=lambda i: fits[i])
    best     = pop[best_idx][:]
    best_fit = fits[best_idx]
    history  = [best_fit]

    start = time.time()
    for _ in range(gens):
        new_pop = [best[:]]  # elitism
        while len(new_pop) < pop_size:
            p1 = select(pop, fits, tsize)
            p2 = select(pop, fits, tsize)
            if random.random() < pc:
                c1 = crossover(p1, p2)
                c2 = crossover(p2, p1)
            else:
                c1, c2 = p1[:], p2[:]
            c1 = mutate(c1, pm)
            c2 = mutate(c2, pm)
            new_pop.extend([c1, c2])

        pop = new_pop[:pop_size]
        fits = [fitness(ind, k, D) for ind in pop]

        gen_best_idx = min(range(len(fits)), key=lambda i: fits[i])
        if fits[gen_best_idx] < best_fit:
            best_fit = fits[gen_best_idx]
            best = pop[gen_best_idx][:]
        history.append(best_fit)

        if max_seconds and (time.time() - start > max_seconds):
            break

    runtime = time.time() - start

    # restore global RNG states (keeps outside code deterministic)
    random.setstate(rnd_state)
    np.random.set_state(np_state)

    return {
        "best_distance": best_fit,
        "best_perm": best,
        "history": history,
        "runtime_sec": runtime
    }

## Medium instances:

In [ ]:
def make_instance(name: str, n_customers: int, vehicles: int, seed: int) -> VRPInstance:
    rng = np.random.default_rng(seed)
    depot = (50.0, 50.0)
    customers = [(float(rng.uniform(0, 100)), float(rng.uniform(0, 100))) for _ in range(n_customers)]
    return VRPInstance(name=name, depot=depot, customers=customers, vehicles=vehicles)

def try_build_csv_instance(csv_path: str, name: str, vehicles: int, n_sample: int) -> VRPInstance:
    if not os.path.exists(csv_path):
        return None
    with open(csv_path, "r", newline="") as f:
        r = csv.reader(f)
        header = next(f, None)  # skip header if present
        rows = []
        reader = csv.reader([header] + list(f)) if header else csv.reader(f)
    # re-open cleanly and actually parse:
    with open(csv_path, "r", newline="") as f2:
        rdr = csv.reader(f2)
        hdr = next(rdr, None)  # ignore header row
        rows = [(int(row[0]), float(row[1]), float(row[2])) for row in rdr if row]
    if len(rows) < n_sample:
        raise ValueError(f"{csv_path} has {len(rows)} rows; need at least {n_sample}.")
    random.seed(RANDOM_SEED)
    picked = random.sample(rows, n_sample)
    customers = [(x, y) for (_cid, x, y) in picked]
    depot = (0.0, 0.0)
    return VRPInstance(name=name, depot=depot, customers=customers, vehicles=vehicles)

def build_medium_instances() -> List[VRPInstance]:
    insts = [
        make_instance("Medium-1", 22, 12, seed=201),
        make_instance("Medium-2", 28, 20, seed=202),
    ]
    csv_inst = try_build_csv_instance("customers.csv", "Medium-CSV", vehicles=12, n_sample=24)
    if csv_inst is not None:
        insts.append(csv_inst)
    return insts


## GA Profiles

Following the "exploration vs. exploitation" principle, we define three parameter
profiles (Fast, Balanced, Thorough) to study the trade-off between runtime and
solution quality:

- Small population and few generations, with higher mutation.  
  Produces quick but less stable solutions (useful for prototyping or tight time limits).  

- Medium settings, generally achieving good solutions at reasonable cost.  

- Large population and more generations, with lower mutation.  
  Aims for the best solutions but requires more computation.  

This setup allows us to compare how GA performance changes with different parameter choices.


In [ ]:

param_sets = {
    "MEDIUM": {"pop": 12, "gens": 3, "pc": 0.9, "pm": 0.10, "tsize": 3},
}


In [ ]:
def main():
    out_dir = "outputs/medium_completed"
    os.makedirs(out_dir, exist_ok=True)

    instances = build_medium_instances()

    # Run once per (instance, profile) and reuse for plots
    results_rows = []
    histories: Dict[Tuple[str, str], List[float]] = {}

    for inst in instances:
        D = build_D(inst)
        n_customers = len(inst.customers)
        for pname, p in param_sets.items():
            res = genetic_algorithm(D, inst.vehicles, n_customers, p, seed=RANDOM_SEED)
            results_rows.append({
                "instance": inst.name,
                "params": pname,
                "customers": n_customers,
                "vehicles": inst.vehicles,
                "pop": p["pop"],
                "gens": p["gens"],
                "pc": p["pc"],
                "pm": p["pm"],
                "best_distance": res["best_distance"],
                "runtime_sec": res["runtime_sec"],
                "avg_route_length": res["best_distance"] / max(1, inst.vehicles),
            })
            histories[(inst.name, pname)] = res["history"]

    # Save table
    df_results = pd.DataFrame(results_rows).sort_values(["instance", "params"]).reset_index(drop=True)
    df_results.to_csv(os.path.join(out_dir, "results.csv"), index=False)
    print(df_results)

    # Plots
    for (inst_name, pname), hist in histories.items():
        plt.figure()
        plt.plot(range(len(hist)), hist)
        plt.xlabel("Generation")
        plt.ylabel("Best-so-far distance")
        plt.title(f"Convergence: {inst_name} | {pname}")
        fname = f"{inst_name}_{pname.replace(' ', '_').replace('(', '').replace(')', '')}_convergence.png"
        plt.savefig(os.path.join(out_dir, fname), bbox_inches="tight")
        plt.close()

    print(f"Saved results and plots to {out_dir}/")

if __name__ == "__main__":
    main()

In [ ]:

# Reads 'customers.csv' (id,x,y), samples 24 customers, builds a Medium instance
customers_to_pick = 24
depot = (0.0, 0.0)

with open("customers.csv","r") as f:
    reader = csv.reader(f)
    next(reader)  # skip header
    rows = [(int(r[0]), float(r[1]), float(r[2])) for r in reader if r]

random.seed(42)
picked = random.sample(rows, customers_to_pick)
csv_customers = [(x, y) for (_cid, x, y) in picked]
medium_csv = VRPInstance(name="Medium-CSV", depot=depot, customers=csv_customers, vehicles=12)

print(f"Built {medium_csv.name} with {len(medium_csv.customers)} customers and {medium_csv.vehicles} vehicles.")


In [ ]:

def make_instance(name, n_customers, vehicles, seed):
    rng = np.random.default_rng(seed)
    depot = (50.0, 50.0)
    customers = [(float(rng.uniform(0,100)), float(rng.uniform(0,100))) for _ in range(n_customers)]
    return VRPInstance(name=name, depot=depot, customers=customers, vehicles=vehicles)

instances = [
    make_instance("Medium-1", 22, 12, 201),
    make_instance("Medium-2", 28, 20, 202),
]
all_instances = instances + [medium_csv]


## Run parameters and collect mertics

In [ ]:

rows = []
for inst in all_instances:
    D = build_D(inst)
    n_customers = len(inst.customers)
    for pname, p in param_sets.items():
        res = genetic_algorithm(D, inst.vehicles, n_customers, p, seed=42)
        rows.append({
            "instance": inst.name,
            "params": pname,
            "customers": n_customers,
            "vehicles": inst.vehicles,
            "pop": p["pop"],
            "gens": p["gens"],
            "pc": p["pc"],
            "pm": p["pm"],
            "best_distance": res["best_distance"],
            "runtime_sec": res["runtime_sec"],
            "avg_route_length": res["best_distance"]/max(1, inst.vehicles),
        })
df_results = pd.DataFrame(rows).sort_values(["instance","params"]).reset_index(drop=True)
df_results


## Save results

In [ ]:

os.makedirs("outputs/medium_completed", exist_ok=True)
df_results.to_csv("outputs/medium_completed/results.csv", index=False)

# Convergence plots per (instance, profile)
for inst in all_instances:
    D = build_D(inst)
    n_customers = len(inst.customers)
    for pname, p in param_sets.items():
        res = genetic_algorithm(D, inst.vehicles, n_customers, p, seed=42)
        plt.figure()
        plt.plot(range(len(res["history"])), res["history"])
        plt.xlabel("Generation")
        plt.ylabel("Best-so-far distance")
        plt.title(f"Convergence: {inst.name} | {pname}")
        fname = f"outputs/medium_completed/{inst.name}_{pname.replace(' ','_').replace('(','').replace(')','')}_convergence.png"
        plt.savefig(fname, bbox_inches="tight")
        plt.close()

print("Saved results to outputs/medium_completed/")
